Para la realización de la primera tarea se estará recreando el material otorgado por el profesor Montenegro para que funcione con GPUs y TPUs. En un inicio debemos asegurarnos que estaremos trabajando primero con CPUs, por lo tanto toca acceder a Runtime > Change runtime type > verificar que la opción seleccionada sea "None", esta es la opción para trabajar con CPUs en Google Colab.

Empezamos con una función para leer los datos.

In [1]:
import numpy as np
import os #para interactuar con funcionalidades del sistema operativo
import gzip #comprime y descomprime archivos .zip


def cargarset(ruta, tipo='train'):
  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rutacate:
    etiquetas = np.frombuffer(rutacate.read(), dtype=np.uint8, offset=8)
    
  with gzip.open(ruta_imagenes, 'rb') as rutaimg:
    imagenes = np.frombuffer(rutaimg.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

Ahora, para poder trabajar con nuestro drive, debemos "montarlo", para eso usaremos un servidor remoto de Google Colab. Para poder recrear este ejercicio, con anterioridad debimos haber cargado los archivos encontrados en https://github.com/zalandoresearch/fashion-mnist/tree/master/data/fashion en una carpeta de nuestro drive.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
ruta = 'gdrive/My Drive/TopicosIA/fashion_mnist_data'

#guardamos lo que retorna nuestra función definida en un inicio, en este caso "cargarset"
X_train, Y_train = cargarset(ruta, tipo='train')
X_test, Y_test = cargarset(ruta, tipo='test')

Mounted at /content/gdrive


Ahora, antes de crear el modelo, debemos garantizar que el set de entrenamiento tenga de tamaño un múltiplo de 128, es decir, se reajustarán los datos.

In [3]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1)) #usaremos reshape para asegurarnos que los datos serán imágenes en escala de grises de 28 * 278 píxeles
X_test = np.reshape(X_test,(9984,28,28,1))

Una vez reajustados los datos, podemos proceder a importar Tensorflow en su segunda versión, que además ya incluye a Keras. 
Keras es una biblioteca de Python minimalista para Deep Learning que puede funcionar sobre Theano o TensorFlow. Fue desarrollada con el objetivo de que los modelos de Deep Learning sean tan rápidos y fáciles tanto para la investigación como el desarrollo. https://unipython.com/introduccion-y-como-instalar-keras-anaconda/

In [4]:
%tensorflow_version 2.x 
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

Versión de TensorFlow: 2.8.0


Una vez ya con TensroFlow, procedemos a importar la librerpia Keras. Primero se crea un contenedor del modelo con sequential, y luego se agregan las tres capas convolucionales. La diferencia entre estas 3 capas son su cantidad de filtros.

In [5]:
modelo = tf.keras.models.Sequential()
modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu')) #64 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu')) #128 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu')) #256 filtros
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

Después de estas tres capas vamos a aplanar los datos y agregaremos una red neuronal.

In [6]:
modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dense(256)) #inicio con 256 neuronas
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10)) #salida con 10 neuronas
modelo.add(tf.keras.layers.Activation('softmax'))

Ahora, se puede imprimir la información del modelo en la pantalla, un summary

In [7]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

Antes del entrenamiento vamos a definir el optimizador a usar, la funcion de error y la metrica de desempeño.

In [8]:
modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Entrenamiento con CPU.

In [15]:
import timeit #usaremos timeit para almacenar el tiempo de ejecución de cada iteración.

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1) #set de entrenamiento, set de validación, tamaño del lote, número de iteraciones
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu') #almacenando el tiempo

Epoch 1/2
468/468 [==============================] - 658s 1s/step - loss: 0.6730 - accuracy: 0.7825 - val_loss: 0.3805 - val_accuracy: 0.8632
Epoch 2/2
468/468 [==============================] - 716s 2s/step - loss: 0.3896 - accuracy: 0.8630 - val_loss: 0.3457 - val_accuracy: 0.8791


In [16]:
print('Tiempo de entrenamiento ' + str(cpu_time)+ ' segundos')

Tiempo de entrenamiento 1402.8152854450004 segundos


# Entrenamiento con GPU

Para este entrenamiento tenemos que tener en cuenta que toca acceder a Runtime > Change runtime type > seleccionar GPU > guardar. Además, toca correr de nuevo todo el código hasta la definición del optimizador a usar.

In [10]:
import timeit #usaremos timeit para almacenar el tiempo de ejecución de cada iteración.

def entrenamiento_gpu():
  with tf.device('/gpu:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1) #set de entrenamiento, set de validación, tamaño del lote, número de iteraciones
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu') #almacenando el tiempo

Epoch 1/2
468/468 [==============================] - 29s 40ms/step - loss: 0.6825 - accuracy: 0.7824 - val_loss: 0.3737 - val_accuracy: 0.8707
Epoch 2/2
468/468 [==============================] - 19s 40ms/step - loss: 0.3917 - accuracy: 0.8629 - val_loss: 0.3423 - val_accuracy: 0.8732


In [11]:
print('Tiempo de entrenamiento ' + str(gpu_time)+ ' segundos')

Tiempo de entrenamiento 82.99002373799999 segundos


# Entrenamiento con TPU

Para el entrenamiento con TPUs toca ir a Runtime > Change runtime type > seleccionar TPU, pero además toca reconocer la TPU, conectarse a esta, inicializarla y crear un objeto donde almacenaremos el modelo a entrenar. Cabe resaltar que antes de todo esto, de nuevo toca correr todo el código hasta de nuevo, la selección del optimizador.

In [11]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

TPU encontrada  ['10.16.150.122:8470']


In [12]:
tf.config.experimental_connect_to_cluster(tpu) #conectar
tf.tpu.experimental.initialize_tpu_system(tpu) #inicializar
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu) #crear el objeto para poder correr el modelo

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.16.150.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.16.150.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Ahora, ya contando con la TPU reconocida, conectada, inicializada y creado tpu strategy, vamos a almacenar el modelo dentro de un with

In [13]:
with tpu_strategy.scope():
  modelo = tf.keras.models.Sequential()
  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.Flatten())
  modelo.add(tf.keras.layers.Dense(256))
  modelo.add(tf.keras.layers.Activation('elu'))
  modelo.add(tf.keras.layers.Dropout(0.5))
  modelo.add(tf.keras.layers.Dense(10))
  modelo.add(tf.keras.layers.Activation('softmax'))
  modelo.summary()

  modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy']) #definimos optimizador, funcion de error y la métrica

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

Ahora, para proceder con el entrenamiento, convertiremos los datos a formato punto flotante de 32 bits, pues es el requerido por Google Colab para usar entrenamientos con TPUs. Una vez con esa transformación, procedemos a correr el entrenamiento.

In [14]:
import timeit

x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

def entrenamiento_tpu():
  modelo.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 16s 23ms/step - loss: 0.7199 - accuracy: 0.7760 - val_loss: 0.3866 - val_accuracy: 0.8648
Epoch 2/2
468/468 [==============================] - 9s 20ms/step - loss: 0.4023 - accuracy: 0.8583 - val_loss: 0.3077 - val_accuracy: 0.8877


In [15]:
print('Tiempo con TPU: ' + str(tpu_time) + ' segundos')

Tiempo con TPU: 28.522188180000285 segundos


# Conclusión:

Como podemos ver, hay diferencias significativas entre el uso de CPU para entrenamientos (en este caso, fueron 24 minutos de entrenamiento) y el uso de GPU, siendo cuestión de un poco más de un minuto el tiempo necesario para el entrenamiento, y por último con el uso de TPU es un poco menos que medio minuto, aunque es más laborioso de usar este método a comparación de la GPU.